In [0]:
%run ../config/setup

In [0]:
from pyspark.sql.functions import avg, sum, min, max, col

df_silver = spark.table(full_path_silver)



In [0]:
# Table 1: hourly
# Purpose: Identify peak load hours.
df_hourly = df_silver.groupBy("hour_of_day") \
    .agg(
        avg("consumption_kwh").alias("avg_consumption_kwh"),
        min("voltage").alias("min_voltage"),
        max("voltage").alias("max_voltage"),
        avg("global_intensity").alias("avg_current_intensity")
    ).orderBy("hour_of_day")
df_hourly.write.format("delta").mode("overwrite").saveAsTable(full_path_gold_hourly)

# display a few values
display(df_hourly.limit(25))

In [0]:
# Table 2: Daily
# Purpose: Understand day-wise variations & off-peak patterns.
df_daily = df_silver.groupBy(col("datetime").cast("date").alias("date")) \
    .agg(
        sum("consumption_kwh").alias("total_kwh"),
        avg("consumption_kwh").alias("avg_kwh"),
        max("consumption_kwh").alias("peak_load")
    ).orderBy("date")

df_daily.write.format("delta").mode("overwrite").saveAsTable(full_path_gold_daily)

# display a few values
display(df_daily.limit(25))

In [0]:
# Table 3: Sub-Meters
# Purpose: Analyze kitchen, laundry, and water heating consumption.
df_submeter = df_silver.groupBy(col("datetime").cast("date").alias("date")) \
    .agg(
        sum("Sub_metering_1").alias("sub_meter_1_total"),
        sum("Sub_metering_2").alias("sub_meter_2_total"),
        sum("Sub_metering_3").alias("sub_meter_3_total")
    ).orderBy("date")

df_submeter.write.format("delta").mode("overwrite").saveAsTable(full_path_gold_submeter)

# display a few values
display(df_submeter.limit(25))